# Section 3: Select and load a project, then generate the two textual maps of Max and Samanta.
The two maps are saved in the project folder as map1.txt and map2.txt.
Use the Delete button to remove a project.

In [1]:
#solo mappa concettualepartendo da due profili essenzialie una domanda
import os
import gradio as gr
import ollama
import threading
import time
import shutil
import json
from pathlib import Path

# Variabili globali
messages = []
sintesi_raccolte = []
indirizzo=""
modello_max='gemma3:12b'
modello_sam='gemma3:12b'
modello_pol='gemma3:12b'

css = """
button:active {
    background-color: #ff6600 !important;  /* colore di sfondo quando premuto */
    color: white !important;               /* colore del testo quando premuto */
    transform: scale(0.97);                /* leggero effetto di "pressione" */
}
"""

# Carica il percorso salvato
CONFIG_PATH = Path(os.getenv("APPDATA")) / "MaGo" / "config.json"

def load_project_path():
    """Carica il percorso dal file di configurazione"""
    if not CONFIG_PATH.exists():
        return None
    
    with open(CONFIG_PATH, "r") as f:
        config = json.load(f)
    return config.get("project_path")

# Verifica all'avvio
project_path = load_project_path()
if not project_path or not Path(project_path).exists():
    raise RuntimeError(
        "🛑 Percorso non configurato o non valido!\n"
        "Esegui prima il notebook 'installer.ipynb' e imposta la cartella corretta."
    )

# Imposta la directory di lavoro
os.chdir(project_path)
print(f"✔️ Directory di lavoro impostata a: {project_path}")


global project_path


def inizializza_discussione_max(comp1, domanda,lingua):
    global messages
    messages = [ 
    {'role': 'system', 'content': f" Sei Max, esperto in {comp1}. Il tuo compito è analizzare e approfondire la seguente domanda: '{domanda}'."},  
    {'role': 'system', 'content': f"Devi individuare e restituire solo una lista estesa e strutturata degli elementi essenziali per comprendere questa domanda. La lista deve essere articolata nelle seguenti categorie: "},  
    {'role': 'system', 'content': """  
- **Concetti fondamentali**: Principi e teorie chiave, formulati esclusivamente nel tuo ambito di competenza in relazione alla domanda.  
- **Relazioni tra concetti fondamentali: Connessioni tra idee per individuare strutture logiche .
- **Evoluzione storica: come i temi espressi nella domanda sono cambiati nel tempo.
- **Scuole di pensiero o approcci teorici**: Correnti di pensiero rilevanti all'interno della tua disciplina.  
- **Autori e figure chiave**: Studiosi ed esperti che hanno dato contributi significativi nel tuo settore di competenza in relazione alla domanda.  
- **Eventi o esperimenti rilevanti**: Scoperte, test o sviluppi cruciali, esaminati attraverso il tuo punto di vista disciplinare sempre in relazione ai temi contenuti nella domanda. 
- **Dibattiti aperti e controversie: Questioni ancora irrisolte o dibattute. 
- **Ambito interdisciplinare**: Connessioni con altre discipline, interpretate dalla prospettiva del tuo campo disciplinare e in relazione alla domanda.  
    """},  
    {'role': 'system', 'content': f" La tua risposta è completamente in lingua {lingua} ed anche i titoli come esempio *concetti fondamentali* sono in lingua {lingua} e deve contenere solo la lista, senza titolo, senza introduzione e senza conclusione. "},  
]  
  
    
    return f"Discussione inizializzata con: {comp1}, {domanda}"

def inizializza_discussione_sam(comp2, domanda,lingua):
    global messages
    messages = [ 
    {'role': 'system', 'content': f" Sei Samanta, esperta in {comp2}. Il tuo compito è analizzare e approfondire la seguente domanda: '{domanda}'."},  
    {'role': 'system', 'content': f"Devi individuare e restituire solo una lista estesa e strutturata degli elementi essenziali per comprendere questa domanda. La lista deve essere articolata nelle seguenti categorie: "},  
    {'role': 'system', 'content': """  
- **Concetti fondamentali**: Principi e teorie chiave, formulati esclusivamente nel tuo ambito di competenza in relazione alla domanda.  
- **Relazioni tra concetti fondamentali: Connessioni tra idee per individuare strutture logiche .
- **Evoluzione storica: come i temi espressi nella domanda sono cambiati nel tempo.
- **Scuole di pensiero o approcci teorici**: Correnti di pensiero rilevanti all'interno della tua disciplina.  
- **Autori e figure chiave**: Studiosi ed esperti che hanno dato contributi significativi nel tuo settore di competenza in relazione alla domanda.  
- **Eventi o esperimenti rilevanti**: Scoperte, test o sviluppi cruciali, esaminati attraverso il tuo punto di vista disciplinare sempre in relazione ai temi contenuti nella domanda. 
- **Dibattiti aperti e controversie: Questioni ancora irrisolte o dibattute. 
- **Ambito interdisciplinare**: Connessioni con altre discipline, interpretate dalla prospettiva del tuo campo disciplinare e in relazione alla domanda.  
    """},  
    {'role': 'system', 'content': f" La tua risposta è completamente in lingua {lingua} ed anche i titoli come esempio *concetti fondamentali* sono in lingua {lingua} e deve contenere solo la lista, senza titolo, senza introduzione e senza conclusione. "},    
]  
 
    
    return f"Discussione inizializzata con:  {comp2},  {domanda}"

def chat_sam(comp2, domanda,codice,indirizzo,lingua):
   
    global messages
    chat_stato_sam = []

    inizializza_discussione_sam(comp2, domanda,lingua)

    response = ollama.chat(model=modello_sam, messages=messages)
    messages.append({'role': 'assistant', 'content': response['message']['content']})
    #print(f" Samanta: {response['message']['content']}\n")
    chat_stato_sam.append(("Samanta", response['message']['content']))
    comp=response['message']['content']

    base_dir = "progetti/"+codice
    os.makedirs(base_dir, exist_ok=True)
    indirizzo = os.path.join(base_dir)
    
    # Scrivi la risposta di Samanta su un file di testo
    with open(indirizzo + '\\map2.txt', 'w', encoding="utf-8") as file:
        file.write(comp)  # Scrive la risposta nel file

    return comp


def chat_max(comp1, domanda, codice,indirizzo,lingua):

    #global messages
    chat_stato_max = []
    profilo=comp1
   
    inizializza_discussione_max(comp1, domanda,lingua)
   
    response = ollama.chat(model=modello_max, messages=messages)
    messages.append({'role': 'assistant', 'content': response['message']['content']})
    #print(f" Max: {response['message']['content']}\n")
    chat_stato_max.append(("Max", response['message']['content']))

    comp=response['message']['content']

    base_dir = "progetti/"+codice
    os.makedirs(base_dir, exist_ok=True)
    
    indirizzo = os.path.join(base_dir)

    # Scrivi la risposta di Max su un file di testo
    with open(indirizzo + '\\map1.txt', 'w', encoding="utf-8") as file:
        file.write(comp)  # Scrive la risposta nel file
    
    return comp
    #return chat_stato_max


def carica_progetto(nome_progetto):
    """Carica i file esp1.txt, esp2.txt e domanda.txt dalla cartella selezionata."""
    global project_path
    
    project_dir = Path(project_path) / "progetti" / nome_progetto
    
    if not os.path.exists(project_dir):
        return "Errore: Progetto non trovato", "", "", "", ""

    def leggi_file(nome_file):
        path = os.path.join(project_dir, nome_file)
        try:
            with open(path, 'r', encoding='utf-8') as f:
                contenuto = f.read().strip()
                return contenuto if contenuto else ""  # Restituisce stringa vuota se il file è vuoto
        except FileNotFoundError:
            print(f"Attenzione: il file '{nome_file}' non è stato trovato.")
            return ""
        except Exception as e:
            print(f"Errore nella lettura del file '{nome_file}': {e}")
            return ""

    domanda = leggi_file("domanda.txt")
    comp1 = leggi_file("esp1.txt")
    comp2 = leggi_file("esp2.txt")
    lingua=leggi_file("lingua.txt")
    map1=leggi_file("map1.txt")
    map2=leggi_file("map2.txt")


    return comp1, comp2, domanda,nome_progetto,lingua,map1,map2


def recupera_progetto(name):
    filename = f"profili/progetti/{name}.txt"
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as f:
            return f.read()
    else:
        return f"Progetto '{name}' non trovato."




def aggiorna_progetti():
    """Restituisce la lista delle cartelle dei progetti."""
    global project_path
    project_dir = Path(project_path) / "progetti" 
    if not os.path.exists(project_dir):
        os.makedirs(project_dir)  # Crea la directory se non esiste
    return [d for d in os.listdir(project_dir) if os.path.isdir(os.path.join(project_dir, d))]
   

def elimina_progetto(progetto):
    global project_path
    project_dir = Path(project_path) / "progetti" / progetto
    if project_dir.exists():
        shutil.rmtree(project_dir)  # Cancella tutta la cartella del progetto
    nuove_scelte = aggiorna_progetti()
    valore_nuovo = nuove_scelte[0] if nuove_scelte else None
    return gr.update(choices=nuove_scelte, value=valore_nuovo), f"Progetto '{progetto}' cancellato."



        

def avvia_interfaccia():
    with gr.Blocks(css=css) as demo:
        gr.Markdown("# Create Concept Maps of Max and Samanta on the Topic Under Study")
        
        with gr.Row():
            Progetto = gr.Dropdown(label="# Select a project", choices=aggiorna_progetti())
        
        with gr.Row():
            btn_carica = gr.Button("Load Project")   
            btn_delete = gr.Button("Delete Project")
            output_delete = gr.Textbox(label="Deletion Status", interactive=False)

        with gr.Row():
            comp1 = gr.Textbox(label="Expertise of Actor 1 (Max)")
            comp2 = gr.Textbox(label="Expertise of Actor 2 (Samanta)")
        
        with gr.Row():
            domanda = gr.Textbox(label="Main Question", scale=2)
            codice = gr.Textbox(label="Project Name")
            lingua = gr.Textbox(label="Language")
            # btn10 = gr.Button("Save", scale=1, min_width=30)

        indirizzo = gr.State()

  
        with gr.Row():  # Side-by-side output panels
            output1 = gr.Markdown(label="Max's Answer")
            output2 = gr.Markdown(label="Samanta's Answer")
        
        with gr.Row():  # Buttons for analysis and concept maps
            btn1 = gr.Button("Generate Max's Concept Map")
            btn2 = gr.Button("Generate Samanta's Concept Map")
        
        with gr.Row():
            mappa_max = gr.Textbox(label="Max", interactive=False)
            mappa_sam = gr.Textbox(label="Samanta", interactive=False)

 

        btn_delete.click(
        fn=elimina_progetto,
        inputs=[Progetto],
        outputs=[Progetto, output_delete]  # esito è una textbox o label per feedback
)
        
        btn_carica.click(fn=carica_progetto, inputs=[Progetto], outputs=[comp1, comp2, domanda, codice, lingua, mappa_max, mappa_sam])
        btn1.click(
            fn=chat_max,
            inputs=[comp1, domanda, codice, indirizzo, lingua],
            outputs=[mappa_max]
        )
        btn2.click(
            fn=chat_sam,
            inputs=[comp2, domanda, codice, indirizzo, lingua],
            outputs=[mappa_sam]
        )

        
    demo.launch()
if __name__ == "__main__":
    avvia_interfaccia()

✔️ Directory di lavoro impostata a: C:\Users\maian\notebook\py10\MaGo
* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Attenzione: il file 'map1.txt' non è stato trovato.
Attenzione: il file 'map2.txt' non è stato trovato.
